# <오늘 할 것: blog 앱 만들기2>

# 1. 글 작성/수정 합치기
- 1.1 `PostEditView`와 `PostView` 합치기
- 1.2 Post와 Form 객체 합치기  

# 2. 글 조회를 1에 합치기

# 3. 게시판 앱 만들기
- 3.0 틀짜기
- 3.1 조회수 + 이미지
- 3.3 N개 게시판
- 3.4 paging
- 3.5 동적 loading

# 1. 글 작성/수정 합치기

## 1.1 `PostEditView`와 `PostView` 합치기

`blog/views.py`의 `PostEditView`에 `PostView`를 합칠거다.

- `PostEditView`는 글을 수정
- `PostView`는 새로운 글을 작성

하는 클래스였고, 원래는 다음과 같이 작성했었다.

In [ ]:
class PostEditView(View):
    def get(self, request, pk):    #글을 수정하는 것이기 때문에 글을 특정할 pk를 전달받아야한다.
        post = Post.objects.get(pk=pk)
        form = PostForm(initial={'title':post.title, 'text': post.text})
        return render(request, 'blog/edit.html', {'form': form, 'pk': pk})

    def post(self, request, pk):
        form = PostForm(request.POST)
        if form.is_valid():
            post = get_object_or_404(Post, pk=pk)
            post.title = form['title'].value()
            post.text = form['text'].value()
            post.publish()
            return redirect("list")
        return render(request, "blog/edit.html", {"form": form, 'pk': pk})

In [ ]:
class PostView(View):
    def get(self, request):
        return render(request, 'blog/edit.html')

    def post(self, request):
        title = request.POST.get('title')
        text = request.POST.get('text')
        username = request.session['username']
        user = User.objects.get(username=username)
        Post.objects.create(title=title, text=text, author=user)
        return redirect('list')

두 개 클래스를 if문을 활용해서 합칠거다. 바꾸면서 url 시스템을 조금 바꿀 건데,
- pk가 0일 때; 즉 `blog/0/edit/`으로 요청이 들어올 때
    - 신규 글 작성
- pk가 0이 아닐 때; 즉 `blog/1/edit` 등으로 요청이 들어올 때
    - 해당 pk를 id로 갖는 글 수정

으로 보낼 거다.

In [ ]:
class PostEditView(View):
    def get(self, request, pk):    #글을 수정하는 것이기 때문에 글을 특정할 pk를 전달받아야한다.
        if pk == 0:            #pk가 0일 경우
            form = PostForm()  #빈 폼 객체를 만든다.
        else:
            post = Post.objects.get(pk=pk)
            form = PostForm(initial={'title':post.title, 'text': post.text})
        return render(request, 'blog/edit.html', {'form': form})

    def post(self, request, pk):
        form = PostForm(request.POST)
        if form.is_valid():
            if pk == 0:    #pk가 0일 경우 유저정보 가져와서 새로 생성
                username = request.session["username"]
                user = User.objects.get(username=username)
                Post.objects.create(title=form['title'].value(), text=form['text'].value(), author=user)
            else :        #pk가 0이 아닐 경우 업데이트 (유저정보는 바뀌지 않으니까 그냥 둠)
                post = get_object_or_404(Post, pk=pk)
                post.title = form['title'].value()
                post.text = form['text'].value()
                post.publish()
            return redirect("list")
        return render(request, "blog/edit.html", {"form": form})

`blog/edit.html`

`action` 인자를 없앤다 --> 자기 자신을 호출한다.

- 즉 `blog/0/edit`으로 들어왔으면 그대로 `blog/0/edit`을 보여준다.
- 그렇기 때문에 위의 `blog/views.py`에서 `render` 함수에 더 이상 pk를 context 정보에 포함하지 않아도 된다. 어차피 자기자신을 호출할 거기 때문에 템플릿을 띄울 때 pk 정보가 필요하지 않다.

In [ ]:
<form method=post>

   {% csrf_token %}
   {{ form.as_p }}

   <input type="submit" value="작성" >
    
</form>

잘 합쳐졌다.

그런데 보완할 수 있다.

## 1.2 Post와 Form 객체 합치기

 `Post`객체를 통해 `blog/models.py`의 모델 데이터를 가져와서 `Form`에 저장하여 띄우는 것이 논리적으로는 유기적으로 연관되지만, 물리적으로는 서로 연관성이 없기 때문에, 둘을 매번 코드로 연결시켜주어야했다.

예를 들면 이 부분이 그렇다.

In [ ]:
post = Post.objects.get(pk=pk)
form = PostForm(initial={'title':post.title, 'text': post.text})

`Post`를 가져온 뒤에 `.title`, `.text`를 통해서 각각의 속성에 접근하여 별도로 저장해주는 방식으로 `form` 객체에 저장하는 방식이다.

이게 코드 길이를 많이 잡아먹고 또 귀찮기도 하는데, 해결할 방법이 있다.

### 바로 `PostForm` 클래스를 다시 정의해주는 것이다.

In [ ]:
from django import forms
from blog.models import Post

In [ ]:
class PostForm(forms.ModelForm):
    class Meta:
        model = Post
        fields = ['title', 'text']   #모델 데이터에서 가져올 필드를 지정한다.

django의 모델과 폼을 합쳐주는 `forms.ModelForm` 객체를 상속받아서 `PostForm` 클래스를 정의했다.

이 때 `Meta` 클래스의 
- `model` 에 `blog/models.py`에 정의한 `Post` 객체를 
- `fields` 에는 `Post` 객체에서 활용할 필드인 `title`과 `text`를 

지정해준다.

변경사항에 알맞게 `blog/views.py`의 `PostEditView` 클래스도 다음과 같이 수정해준다.

In [ ]:
class PostEditView(View):
    def get(self, request, pk):   
        if pk == 0: 
            form = PostForm() 
        else:
            post = Post.objects.get(pk=pk)
            form = PostForm(instance=post)
        return render(request, 'blog/edit.html', {'form': form})

    def post(self, request, pk):
        username = request.session['username']
        user = User.objects.get(username=username)
        if pk == 0:
            form = forms.PostForm(request.POST)
        else:
            post = get_object_or_404(Post, pk=pk)
            form = PostForm(request.POST, instance=post)

        if form.is_valid():
            post = form.save(commit=False)
            if pk == 0:
                post.author = user
                post.save()
            else:
                post.publish()
            return redirect('list')
        return render(request, "blog/edit.html", {'form': form})

가장 두드러지는 변화는 

# 2. 글 조회를 1에 합치기

글을 조회하는 `list`와 `detail`도 `PostEditView`에 합칠 수 있다.

글 id(pk)를 입력받아서 특정 작업(조회, 작성, 수정)을 수행한다는 동작 방식이 동일하기 때문이다.

마찬가지로 url 동작 방식을 다음과 같이 바꿀 거다.

- `blog/0/list`: 글 목록 출력
- `blog/5/detail`: id 5인 글 조회
- `blog/0/add`: 신규 글 작성
- `blog/5/edit`: id 5인 글 수정

먼저 위 내용을 반영해서 `blog/urls.py`를 수정한다.

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    ...
    path('<int:pk>/<mode>/', views.PostEditView.as_view(), name='edit'), #.as_view(): class를 view로 호출하기
    ...
]

글 id를 `<int:pk>`에, 수행할 작업을 `<mode>`에 입력받아서 일단은 전부 다 `PostEditView`로 연결해줄 거다.

이제 이걸 `blog/views.py`의 `PostEditView`에서 받아서 다음과 같이 처리할 거다.



# 3. 게시판 앱 만들기

## 3.0 틀짜기

사실상 blog 앱에 부가 기능을 추가한 앱이다. 그래서 일단 blog 앱에서 대부분의 파일을 복사해온다.
### 앱 생성
$ `python manage.py startapp myboard` 를 통해서 새로운 앱을 생성한 뒤에

- `blog` 폴더에서
    - `admin.py`
    - `forms.py`
    - `models.py`
    - `urls.py`
    - `views.py`  를,

- `templates` 폴더에서 모든 템플릿을 복사해서 myblog에 붙여넣어(override)준다.

### 초기 세팅
루트(`mysite`)의 `mysite/settings.py`와 `mysite/urls.py`에 새로 만든 앱을 등록해준다.

In [ ]:
#settings.py
INSTALLED_APPS = [
    ...
    'myboard',
]

In [ ]:
#urls.py
urlpatterns = [
    ...
    path('myboard/', include('myboard.urls')),
]


이제 본격적으로 게시판 앱을 만든다.

다시 짚고 넘어가자면, django에서 앱을 개발하는 순서는
1. Model 만들기
2. Form 만들기
3. View 만들기

순으로 이루어진다.

`myboard/models.py`에서 `Board`라는 이름의 모델 객체를 생성한다.

In [ ]:
from django.db import models
from django.utils import timezone


# Create your models here.

class Board(models.Model):
    author = models.ForeignKey('auth.User', on_delete=models.CASCADE)   #auth.User를 지울 때 해당 User가 작성한 Post도 지우도록 설정
    title = models.CharField(max_length=200)
    text = models.TextField()  # 글자수에 제한 없는 텍스트
    created_date = models.DateTimeField(
        default=timezone.now)  # 날짜와 시간
    cnt = models.IntegerField(default=0)
    image = models.CharField(max_length=200, null=True, blank=True)
    category = models.CharField(max_length=10, default='common')

    def __str__(self):
        return self.title

- 조회수를 저장할 `cnt`와
- 이미지를 저장할 `image`와
- 카테고리를 저장할 `category`를 추가적으로 정의해주었다. 

그리고 `published_date`는 삭제했다.

`myboard/admin.py`에 위에서 생성한 `Board` 테이블을 등록해준다.

In [ ]:
from django.contrib import admin
from . import models

# Register your models here.
admin.site.register(models.Board)

이 단계에서 `migrate`를 해주어야 한다!

다시 짚고 넘어가자면
- $`python manage.py makemigrations`
- $`python manage.py migrate`

로 `migrate`를 수행할 수 있다.

이제 1. Model 만들기까지는 끝났고, 2. Form을 만들면 된다.

`myblog/forms.py`에서 복사해온 `myboard/forms.py`를 수정한다.

In [ ]:
from django import forms
from . import models
from django.forms import ValidationError

def validator(value):
    if len(value) < 5: raise ValidationError('길이가 너무 짧아요')

class BoardForm(forms.ModelForm):
    class Meta:
        model = models.Board
        fields = ['title', 'text', 'category'] 
        
    def __init__(self, *args, **kwargs):
        super(BoardForm, self).__init__(*args, **kwargs)
        self.fields['title'].validators = [validator]

정말 단순화하면 `Form`이라고 되어 있던 걸 `Board`로 바꿔주면된다.

그리고 `myboard/forms.py`를 만드는 이유를 다시 한번 간단히 짚고 넘어가면, `myboard/models.py`에 생성한 `Board` 모델(테이블)에서 `myboard/views.py`에서 조작할 필드만 선택적으로 가져와서 다시 객체화하기 위해서다.

이제 2. Form 만들기까지 완료했고, 가장 코딩이 많은 3. View 만들기만 하면 된다.

`myboard/views.py`를 수정한다.

In [ ]:
from django.shortcuts import render, get_object_or_404, redirect
from django.http import HttpResponse
from django.views.generic import View
from django.contrib.auth.models import User
from . import forms
from . import models

# Create your views here.
class BoardView(View):
    def get(self, request, pk, mode):
        if mode == 'add':
            form = forms.BoardForm()
        elif mode == 'list':
            username = request.session['username']
            user = User.objects.get(username=username)
            data = models.Board.objects.all().filter(author=user)
            context = {"data": data, "username": username}
            return render(request, "myboard/list.html", context)
        elif mode == 'detail':
            p = get_object_or_404(models.Board, pk=pk)
            return render(request, 'myboard/detail.html', {'d': p})
        elif mode == 'edit':
            post = get_object_or_404(models.Board, pk=pk)
            form = forms.BoardForm(instance=post)
        else:
            return HttpResponse("error page")
        return render(request, 'myboard/edit.html', {'form': form})

    def post(self, request, pk, mode):
        username = request.session['username']
        user = User.objects.get(username=username)
        if pk == 0:
            form = forms.BoardForm(request.POST)
        else:
            post = get_object_or_404(models.Board, pk=pk)
            form = forms.BoardForm(request.POST, instance=post)

        if form.is_valid():
            post = form.save(commit=False)
            if pk == 0:
                post.author = user
            else:
                post.cnt += 1
            post.save()
            return redirect('myboard', 0, 'list')
        return render(request, "myboard/edit.html", {'form': form})

- `Form` --> `Board`
- `Post` --> `Board`
- `blog` --> `myboard`

로 일단 기계적으로 다 바꿔준다.

템플릿들도 마찬가지의 작업을 해주고, 서버를 실행해보면서 에러를 핸들링해나가면 된다.

이제 본격적으로 부가적인 기능들을 만들어보겠다.

## 3.1 조회수 + 이미지

### 조회수 띄우기

쉽다. 그냥 `myboard/views.py`의 `BoardView` 클래스에서 `detail`이 호출될 때마다 `cnt` 값이 하나씩 늘어나도록 처리해주면 된다.

In [ ]:
class BoardView(View):
    def get(self, request, pk, mode):
        if mode == 'add':
            form = forms.BoardForm()
        elif mode == 'list':
            username = request.session['username']
            user = User.objects.get(username=username)
            data = models.Board.objects.all().filter(author=user)
            context = {"data": data, "username": username}
            return render(request, "myboard/list.html", context)
        elif mode == 'detail':
            p = get_object_or_404(models.Board, pk=pk)
            p.cnt += 1   #바로 여기!
            p.save()
            return render(request, 'myboard/detail.html', {'d': p})
        ...

그리고 여러 템플릿에서 `.cnt`로 해당 글의 조회수에 접근해서 조회수를 띄울 수 있다.

### 이미지 업로드
이미지 업로드는 예전에 했던 코드를 참조하면 된다.

먼저 템플릿을 만든다.

이 때 중요한 건, 이미지를 업로드하는 코드를 기존 `edit.html`에 포함시키는 게 아니라 별도의 템플릿으로 만들어서 `edit.html`에 끼워넣어 주어야 한다.

그 이유는
- 코드를 분리해서 관리해서 오는 이점인 업데이트에 강력하다는 장점을 갖는다.
- 글 수정 시 이미지를 다시 업로드하지 않아도 된다.

새 글 작성/기존 글 수정이 이루어지는 `edit.html`을 extend하는 `image.html`을 다음과 같이 작성한다.

In [ ]:
{% extends 'myboard/edit.html' %}

{% block content %}
<input type="file" name="file1"/>
{% endblock %}

세상 간단하다.

그리고 기존의 `edit.html`에 이미지 업로드란을 넣고 싶은 곳에 
- `{% block content %}`
- `{% endblock %}`

을 넣어준다.

이제 템플릿은 끝났고, `myboard/views.py`를 수정할 차례다. view는 좀 더 복잡하다.

여기서 중요한 건
- 실제 파일은 base 디렉토리의 `static` 폴더에 저장하고
- **데이터베이스에는 파일 경로**를 저장한다는 것이다.

`BoardView` 클래스의 `post` 함수를 다음과 같이 수정하면 된다.

In [ ]:
    def post(self, request, category, pk, mode):
        username = request.session['username']
        user = User.objects.get(username=username)
        if pk == 0:
            form = forms.BoardForm(request.POST)
        else:
            post = get_object_or_404(models.Board, pk=pk)
            form = forms.BoardForm(request.POST, instance=post)

        if form.is_valid():
            post = form.save(commit=False)
            if pk == 0:
                post.author = user
                post.category=category
                file = request.FILES.get('file1')    #여기부터
                filename = file._name
                fp = open(settings.BASE_DIR + "/static/" + filename, "wb")
                for chunk in file.chunks():
                    fp.write(chunk)
                fp.close()
                post.image = filename                #여기까지다
            post.save()
            return redirect('myboard', category, 0, 'list')
        return render(request, "myboard/upload.html", {'form': form})

## 3.3 N개 게시판

ORM이 정말 간편하다는 것을 실감할 수 있는 부분이다.

1. `myboard/urls.py`에 `<category>`를 추가하고
2. `myboard/views.py`의 `BoardView`의 함수들에 `category` 몇 개만 추가하고
3. 여기에 맞춰서 html 템플릿 파일들을 약간 수정해주면 끝이다.

In [ ]:
#urls.py
from django.urls import path
from . import views
from django.shortcuts import redirect

urlpatterns = [
    path('<category>/<int:pk>/<mode>/', views.BoardView.as_view(), name='myboard'),
    path('', lambda request: redirect('myboard', 'common', 0, 'list')),
    #이건 'myboard/'로 들어왔을 때 'myboard/common/0/list'로 리다이렉팅 시켜주기 위한 코드다.
]

In [ ]:
#views.py
class BoardView(View):
    def get(self, request, category, pk, mode):  #여기에 category 추가
        if mode == 'add':
            form = forms.BoardForm()
        elif mode == 'list':
            username = request.session['username']
            user = User.objects.get(username=username)
            data = models.Board.objects.all().filter(category=category)
            context = {"data": data, "category": category, "username": username}
            return render(request, "myboard/list.html", context)
        elif mode == 'detail':
            p = get_object_or_404(models.Board, pk=pk)
            p.cnt += 1
            p.save()
            return render(request, 'myboard/detail.html', {'d': p})
        elif mode == 'edit':
            post = get_object_or_404(models.Board, pk=pk)
            form = forms.BoardForm(instance=post)
        else:
            return HttpResponse("error page")
        return render(request, 'myboard/edit.html', {'form': form})

    def post(self, request, category, pk, mode):   #여기에도 category 추가
        username = request.session['username']
        user = User.objects.get(username=username)
        if pk == 0:
            form = forms.BoardForm(request.POST)
        else:
            post = get_object_or_404(models.Board, pk=pk)
            form = forms.BoardForm(request.POST, instance=post)

        if form.is_valid():
            post = form.save(commit=False)
            if pk == 0:
                post.author = user
                post.category=category   #주의!!
            post.save()
            return redirect('myboard', category, 0, 'list')
        return render(request, "myboard/edit.html", {'form': form})


살짝만 주의할 점은, `blog/models.py`를 정의할 때 `category` 필드의 default로 `common`을 지정해두었기 때문에, 새 글을 작성해서 저장하기 전에 `category`를 해당 글이 속한 카테고리로 재지정해줘야 한다.

이것만 빼면 이렇게 간편하게 N개의 게시판을 구현할 수 있다.

## 3.4 paging

## 3.5 동적 loading